# Loxone API Testing Notebook

This notebook allows you to test the exact API calls being made to your Loxone Miniserver.

## Setup

First, install the required package if you haven't already:
```bash
npm install ws
```

## Configuration

Update these values with your Miniserver details:

In [ ]:
// Your Miniserver configuration
const MINISERVER_HOST = '192.168.1.77';  // Replace with your Miniserver IP
const MINISERVER_PORT = 80;
const USERNAME = 'your_username';  // Replace with your username
const PASSWORD = 'your_password';  // Replace with your password

// The UUID of your LightControllerV2 from the web config
const LIGHT_UUID = '0f87c78b-01f9-ddc5-ffff403fb0c34b9e';  // Replace with your actual UUID

console.log('Configuration loaded');

## Test 1: HTTP Authentication and Structure File

This tests if we can authenticate and get the structure file via HTTP (this is working based on the web config).

In [ ]:
const http = require('http');

function testHTTPAuth() {
    return new Promise((resolve, reject) => {
        const auth = Buffer.from(`${USERNAME}:${PASSWORD}`).toString('base64');
        
        const options = {
            hostname: MINISERVER_HOST,
            port: MINISERVER_PORT,
            path: '/data/LoxAPP3.json',
            method: 'GET',
            headers: {
                'Authorization': `Basic ${auth}`
            },
            timeout: 10000
        };
        
        console.log('Testing HTTP authentication...');
        console.log(`URL: http://${MINISERVER_HOST}:${MINISERVER_PORT}/data/LoxAPP3.json`);
        
        const req = http.request(options, (res) => {
            console.log(`Response status: ${res.statusCode}`);
            
            if (res.statusCode === 200) {
                let data = '';
                res.on('data', chunk => data += chunk);
                res.on('end', () => {
                    try {
                        const json = JSON.parse(data);
                        console.log('✅ HTTP authentication successful!');
                        console.log(`Found ${Object.keys(json.controls || {}).length} controls`);
                        resolve(json);
                    } catch (e) {
                        reject(e);
                    }
                });
            } else {
                reject(new Error(`HTTP ${res.statusCode}`));
            }
        });
        
        req.on('error', reject);
        req.on('timeout', () => reject(new Error('Timeout')));
        req.end();
    });
}

await testHTTPAuth();

## Test 2: WebSocket Connection and Authentication

This tests the WebSocket connection that we use for sending commands.

In [ ]:
const WebSocket = require('ws');
const crypto = require('crypto');

let ws = null;

function connectWebSocket() {
    return new Promise((resolve, reject) => {
        const wsUrl = `ws://${MINISERVER_HOST}:${MINISERVER_PORT}/ws/rfc6455`;
        console.log(`Connecting to: ${wsUrl}`);
        
        ws = new WebSocket(wsUrl);
        
        ws.on('open', () => {
            console.log('✅ WebSocket connected');
            resolve();
        });
        
        ws.on('error', (err) => {
            console.error('❌ WebSocket error:', err.message);
            reject(err);
        });
        
        ws.on('message', (data) => {
            console.log('Received:', data.toString());
        });
    });
}

function authenticateWebSocket() {
    return new Promise((resolve, reject) => {
        console.log('Authenticating over WebSocket...');
        
        const messageHandler = (data) => {
            const message = data.toString();
            console.log('Auth response:', message);
            
            try {
                const json = JSON.parse(message);
                
                if (json.LL && json.LL.control === 'jdev/sys/keyexchange') {
                    const key = json.LL.value;
                    console.log('Received key:', key);
                    
                    // Hash password with key
                    const pwHash = crypto.createHash('sha1')
                        .update(`${PASSWORD}:${key}`)
                        .digest('hex')
                        .toUpperCase();
                    
                    const userHash = crypto.createHash('sha1')
                        .update(`${USERNAME}:${pwHash}`)
                        .digest('hex')
                        .toUpperCase();
                    
                    const authCmd = `authenticate/${userHash}`;
                    console.log('Sending auth command:', authCmd);
                    ws.send(authCmd);
                } else if (json.LL && json.LL.control === 'authenticate') {
                    if (json.LL.value === '200') {
                        console.log('✅ Authentication successful!');
                        ws.off('message', messageHandler);
                        resolve();
                    } else {
                        console.error('❌ Authentication failed:', json.LL.value);
                        reject(new Error('Auth failed'));
                    }
                }
            } catch (e) {
                // Not JSON, ignore
            }
        };
        
        ws.on('message', messageHandler);
        
        // Request key exchange
        console.log('Requesting key exchange...');
        ws.send('jdev/sys/keyexchange');
        
        setTimeout(() => reject(new Error('Auth timeout')), 10000);
    });
}

await connectWebSocket();
await authenticateWebSocket();

## Test 3: Send Control Command (The Actual API Call)

This is the exact command we're trying to send from the Stream Deck plugin.

### Command Format for LightControllerV2:
According to Loxone documentation, LightControllerV2 can accept these commands:
- `plus` - Toggle on/off
- `On` - Turn on
- `Off` - Turn off
- Numeric value 0-100 - Set brightness percentage

In [ ]:
function sendCommand(uuid, command) {
    return new Promise((resolve, reject) => {
        if (!ws || ws.readyState !== WebSocket.OPEN) {
            reject(new Error('WebSocket not connected'));
            return;
        }
        
        const cmdString = `jdev/sps/io/${uuid}/${command}`;
        
        console.log('═══════════════════════════════════════');
        console.log('SENDING COMMAND:');
        console.log(`  WebSocket URL: ws://${MINISERVER_HOST}:${MINISERVER_PORT}/ws/rfc6455`);
        console.log(`  UUID: ${uuid}`);
        console.log(`  Command: ${command}`);
        console.log(`  Full command string: ${cmdString}`);
        console.log('═══════════════════════════════════════');
        
        const messageHandler = (data) => {
            const message = data.toString();
            console.log('Response:', message);
            
            try {
                const json = JSON.parse(message);
                if (json.LL && json.LL.Code === '200') {
                    console.log('✅ Command accepted by Miniserver');
                } else {
                    console.log('Response data:', json);
                }
            } catch (e) {
                // Not JSON
            }
        };
        
        ws.on('message', messageHandler);
        
        ws.send(cmdString);
        
        setTimeout(() => {
            ws.off('message', messageHandler);
            resolve();
        }, 2000);
    });
}

// Test the 'plus' command (toggle)
await sendCommand(LIGHT_UUID, 'plus');

## Test 4: Try Alternative Commands

Let's try other commands that LightControllerV2 might accept:

In [ ]:
// Try 'On' command
console.log('\n--- Testing "On" command ---');
await sendCommand(LIGHT_UUID, 'On');

await new Promise(resolve => setTimeout(resolve, 3000));

// Try 'Off' command
console.log('\n--- Testing "Off" command ---');
await sendCommand(LIGHT_UUID, 'Off');

await new Promise(resolve => setTimeout(resolve, 3000));

// Try numeric value (100 = full brightness)
console.log('\n--- Testing "100" command (full brightness) ---');
await sendCommand(LIGHT_UUID, '100');

## Test 5: Try HTTP Command (Alternative Method)

Some Loxone Gen 1 Miniservers might prefer HTTP commands over WebSocket for controls.

In [ ]:
function sendHTTPCommand(uuid, command) {
    return new Promise((resolve, reject) => {
        const auth = Buffer.from(`${USERNAME}:${PASSWORD}`).toString('base64');
        const path = `/dev/sps/io/${uuid}/${command}`;
        
        const options = {
            hostname: MINISERVER_HOST,
            port: MINISERVER_PORT,
            path: path,
            method: 'GET',
            headers: {
                'Authorization': `Basic ${auth}`
            },
            timeout: 5000
        };
        
        console.log('═══════════════════════════════════════');
        console.log('SENDING HTTP COMMAND:');
        console.log(`  URL: http://${MINISERVER_HOST}:${MINISERVER_PORT}${path}`);
        console.log(`  UUID: ${uuid}`);
        console.log(`  Command: ${command}`);
        console.log('═══════════════════════════════════════');
        
        const req = http.request(options, (res) => {
            console.log(`Response status: ${res.statusCode}`);
            
            let data = '';
            res.on('data', chunk => data += chunk);
            res.on('end', () => {
                console.log('Response body:', data);
                if (res.statusCode === 200) {
                    console.log('✅ HTTP command successful');
                    resolve(data);
                } else {
                    reject(new Error(`HTTP ${res.statusCode}`));
                }
            });
        });
        
        req.on('error', reject);
        req.on('timeout', () => reject(new Error('Timeout')));
        req.end();
    });
}

console.log('\n--- Testing HTTP command with "On" ---');
await sendHTTPCommand(LIGHT_UUID, 'On');

## Cleanup

Close the WebSocket connection:

In [ ]:
if (ws) {
    ws.close();
    console.log('WebSocket closed');
}

## Summary

This notebook tests:

1. **HTTP Authentication** - Getting structure file (✅ known working)
2. **WebSocket Connection** - Establishing WS connection
3. **WebSocket Authentication** - Authenticating over WS
4. **Control Commands via WebSocket** - `jdev/sps/io/{uuid}/{command}`
5. **Control Commands via HTTP** - `http://{host}/dev/sps/io/{uuid}/{command}` (alternative)

### Expected Results:

- If WebSocket commands work: You should see response with Code 200 and the light should toggle
- If HTTP commands work better: The HTTP test should return success and control the light

### Next Steps:

Based on which method works, we'll update the plugin to use the correct API format.